<a href="https://colab.research.google.com/github/ditarnnt/Telecom-Predic-Churn/blob/main/ML_Orange_preprocess_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training: Run all

Evaluating: Run dari awal sampai feature selection, lalu run 'Evaluation'

# Starting steps

In [ ]:
from google.colab import drive
import pandas as pd #pandas
import numpy as np #numpy
import re
from sklearn.preprocessing import LabelEncoder, OneHotEncoder #encoding data kategorikal
from sklearn.preprocessing import StandardScaler #standardization
from sklearn.model_selection import train_test_split #train test split
from sklearn.ensemble import RandomForestClassifier
import time
import pickle

!pip install ipython-autotime
%load_ext autotime

rand_state = 101 #hyperparameter search argument
niter = 125 #hyperparameter search argument
kfold = 3 #hyperparameter search argument

time: 1.87 ms (started: 2021-05-11 10:49:22 +00:00)


In [ ]:
drive.mount('/content/drive')
dataset = pd.read_csv('/content/drive/MyDrive/TubesAI_folder/Dataset/orange_preprocessed.csv')  
print (dataset)

Mounted at /content/drive
      Account length  Number vmail messages  ...  International plan  Churn
0              128.0                   25.0  ...                 0.0    0.0
1              107.0                   26.0  ...                 0.0    0.0
2              137.0                    0.0  ...                 0.0    0.0
3               84.0                    0.0  ...                 1.0    0.0
4               75.0                    0.0  ...                 1.0    0.0
...              ...                    ...  ...                 ...    ...
3328           114.0                   26.0  ...                 0.0    0.0
3329           106.0                   29.0  ...                 0.0    0.0
3330            60.0                    0.0  ...                 0.0    0.0
3331           159.0                    0.0  ...                 0.0    0.0
3332           184.0                    0.0  ...                 1.0    0.0

[3333 rows x 18 columns]
time: 20.8 s (started: 2021-05-11 10

In [ ]:
def select_features(df,selected_features):
  '''
  Fungsi untuk memilih fitur pada dataset.
  Menggunakan RegEx untuk seleksi fitur yang di-encode secara OneHot Encoding.
  Ketentuan seleksi fitur OneHotEncoding: Fitur harus punya prefix yang sesuai dengan nama fitur.
  Argumen:
  df - dataset yang akan diseleksi fitur-fiturnya
  selected_features - list berisi fitur-fitur yang akan dipilih
  Output:
  df_out - dataset yang berisi fitur-fitur pada selected_features.
  '''
  df_out = pd.DataFrame(index=df.index.tolist())
  cols = []
  for feature in selected_features:
    r = re.compile(feature) # Read Note
    cols = cols + list(filter(r.match,df.columns.tolist()))
  df_out = df[cols]
  return df_out

time: 8.03 ms (started: 2021-05-11 10:49:43 +00:00)


In [ ]:
y = dataset['Churn']
X = dataset.drop(['Churn'], axis = 1)

time: 4.34 ms (started: 2021-05-11 10:49:43 +00:00)


In [ ]:
print(f'X: {X.shape}')

X: (3333, 17)
time: 3.19 ms (started: 2021-05-11 10:49:43 +00:00)


In [ ]:
X_train , X_test, y_train, y_test = train_test_split(X, y , test_size = 0.200 , random_state = rand_state)

time: 12.2 ms (started: 2021-05-11 10:49:43 +00:00)


In [ ]:
print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')

X_train: (2666, 17)
y_train: (2666,)
X_test: (667, 17)
y_test: (667,)
time: 6.34 ms (started: 2021-05-11 10:49:43 +00:00)


In [ ]:
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

time: 3.01 ms (started: 2021-05-11 10:49:43 +00:00)


# Base Model

In [ ]:
classifier = RandomForestClassifier(random_state = rand_state)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

time: 758 ms (started: 2021-05-11 10:49:43 +00:00)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print (classification_report(y_test, y_pred))
print (confusion_matrix(y_test, y_pred))
print (accuracy_score(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.96      0.99      0.97       573
           1       0.91      0.73      0.81        94

    accuracy                           0.95       667
   macro avg       0.93      0.86      0.89       667
weighted avg       0.95      0.95      0.95       667

[[566   7]
 [ 25  69]]
0.952023988005997
time: 21.7 ms (started: 2021-05-11 10:49:44 +00:00)


# Proposed Model: Feature Selection

In [ ]:
'''
             Total day minutes
 0.0774    5 Total day charge
 0.05009  15 Customer service calls
 0.03688  17 International plan
 0.00822  16 Voice mail plan
 0.00822   2 Number vmail messages
 0.00729  13 Total intl calls
 0.00674  12 Total intl minutes
 0.00674  14 Total intl charge
 0.00542   8 Total eve charge
 0.00542   6 Total eve minutes
 0         4 Total day calls
 0         9 Total night minutes
 0         7 Total eve calls
 0        10 Total night calls
 0        11 Total night charge
 0         1 Account length
 '''

'\n             Total day minutes\n 0.0774    5 Total day charge\n 0.05009  15 Customer service calls\n 0.03688  17 International plan\n 0.00822  16 Voice mail plan\n 0.00822   2 Number vmail messages\n 0.00729  13 Total intl calls\n 0.00674  12 Total intl minutes\n 0.00674  14 Total intl charge\n 0.00542   8 Total eve charge\n 0.00542   6 Total eve minutes\n 0         4 Total day calls\n 0         9 Total night minutes\n 0         7 Total eve calls\n 0        10 Total night calls\n 0        11 Total night charge\n 0         1 Account length\n '

time: 5.52 ms (started: 2021-05-11 10:49:44 +00:00)


In [ ]:
'''
selected_features = ['Total day minutes','Total day charge', 'Customer service calls', 'International plan', 'Voice mail plan',
                     'Number vmail messages', 'Total intl calls', 'Total intl minutes', 'Total intl charge',
                     'Total eve charge', 'Total eve minutes', 'Total day calls', 'Total night minutes', 
                     'Total eve calls', 'Total night calls','Total night charge', 'Account length', 'Churn']
'''

selected_features = ['Total day minutes','Total day charge', 'Customer service calls', 'International plan', 'Voice mail plan',
                     'Number vmail messages', 'Total intl calls', 'Total intl minutes', 'Total intl charge',
                     'Total eve charge', 'Total eve minutes', 'Churn']

time: 2.69 ms (started: 2021-05-11 10:49:44 +00:00)


In [ ]:
df_out = select_features(dataset, selected_features)
y = df_out['Churn']
X = df_out.drop(['Churn'], axis = 1)
X_train , X_test, y_train, y_test = train_test_split(X, y , test_size = 0.200 , random_state = rand_state)

time: 21.1 ms (started: 2021-05-11 10:49:44 +00:00)


# Proposed Model: Hyperparameter Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.model_selection import cross_val_score

random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': list(np.linspace(10, 600, 8, dtype = int)) + [None],
               'max_features': ['auto', 'sqrt','log2', None],
               'min_samples_leaf': [4, 6, 8, 12],
               'min_samples_split': [5, 7, 10, 14],
               'n_estimators': list(np.linspace(50, 300, 8, dtype = int))}

clf = RandomForestClassifier()
#model = GridSearchCV(clf, random_search,
#                               cv = 3, verbose= 5, n_jobs = -1)
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = niter, 
                               cv = kfold, verbose= 5, random_state= rand_state, n_jobs = -1)
model.fit(X_train,y_train) 

Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   59.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:  4.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

time: 4min 7s (started: 2021-05-11 10:49:44 +00:00)


In [ ]:
predictionforest = model.best_estimator_.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc = accuracy_score(y_test,y_pred)
acc1 = accuracy_score(y_test,predictionforest)

[[566   7]
 [ 23  71]]
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.97       573
         1.0       0.91      0.76      0.83        94

    accuracy                           0.96       667
   macro avg       0.94      0.87      0.90       667
weighted avg       0.95      0.96      0.95       667

time: 61.4 ms (started: 2021-05-11 10:53:51 +00:00)


In [ ]:
print("Base accuracy:",acc*100,"%")
print("Optimized accuracy:",acc1*100,"%")
print("Increase in accuracy:",(acc1-acc)*100,"%")

Base accuracy: 95.2023988005997 %
Optimized accuracy: 95.50224887556223 %
Increase in accuracy: 0.2998500749625266 %
time: 2.53 ms (started: 2021-05-11 10:53:51 +00:00)


# Save Base & Proposed Model

In [ ]:
#ngesave model
runtime = time.strftime("%d-%m-%Y %H:%M:%S")

pkl_filename = "/content/drive/MyDrive/TubesAI_folder/Model_Orange/orange_model_" + runtime + ".pkl"
with open(pkl_filename, 'wb') as file:
  pickle.dump(model, file)

base_filename = "/content/drive/MyDrive/TubesAI_folder/Model_Orange/orange_base_" + runtime + ".pkl"
with open(base_filename, 'wb') as file:
  pickle.dump(classifier, file)

with open("/content/drive/MyDrive/TubesAI_folder/Model_Orange/log_" + runtime + ".txt",'a') as file:
  L = ["base accuracy = "+str(acc*100)+"%\n",
       "opt accuracy = "+str(acc1*100)+"%\n",
       "Increase in accuracy:"+str((acc1-acc)*100)+"%\n",
       str(model.best_estimator_.get_params())]
  file.writelines(L)

time: 76.7 ms (started: 2021-05-11 10:53:51 +00:00)


In [ ]:
model.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 4,
 'min_samples_split': 14,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 264,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

time: 8.99 ms (started: 2021-05-11 10:53:51 +00:00)


# Evaluation

In [ ]:
infile = open('/content/drive/MyDrive/TubesAI_folder/Model_Orange/orange_model_'+runtime+'.pkl','rb')
evalmodel = pickle.load(infile)
infile.close()

evalmodel.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 4,
 'min_samples_split': 14,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 264,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

time: 33.4 ms (started: 2021-05-11 10:53:51 +00:00)


In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score

y_eval = evalmodel.best_estimator_.predict(X_test)

acceval = accuracy_score(y_test,y_eval)*100
preciseval = precision_score(y_test,y_eval)*100
recalleval = recall_score(y_test,y_eval)*100
f1eval = f1_score(y_test,y_eval)*100

print("EVALUATION OF PROPOSED MODEL:\n")

print("Summary:")
print(classification_report(y_test,y_eval))

print("Confusion matrix:")
print(str(confusion_matrix(y_test,y_eval))+"\n")

print("Accuracy:")
print(str(acceval)+"%\n")

print("Precision:")
print(str(preciseval)+"%\n")

print("Recall:")
print(str(recalleval)+"%\n")

print("F1 Score:")
print(str(f1eval)+"%\n")

EVALUATION OF PROPOSED MODEL:

Summary:
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.97       573
         1.0       0.91      0.76      0.83        94

    accuracy                           0.96       667
   macro avg       0.94      0.87      0.90       667
weighted avg       0.95      0.96      0.95       667

Confusion matrix:
[[566   7]
 [ 23  71]]

Accuracy:
95.50224887556223%

Precision:
91.02564102564102%

Recall:
75.53191489361703%

F1 Score:
82.55813953488371%

time: 93 ms (started: 2021-05-11 10:53:51 +00:00)


In [ ]:
accpred = accuracy_score(y_test,y_pred)*100
precispred = precision_score(y_test,y_pred)*100
recallpred = recall_score(y_test,y_pred)*100
f1pred = f1_score(y_test,y_pred)*100

print("EVALUATION OF BASE MODEL:\n")

print("Summary:")
print(classification_report(y_test,y_pred))

print("Confusion matrix:")
print(str(confusion_matrix(y_test,y_pred))+"\n")

print("Accuracy:")
print(str(accpred)+"%\n")

print("Precision:")
print(str(precispred)+"%\n")

print("Recall:")
print(str(recallpred)+"%\n")

print("F1 Score:")
print(str(f1pred)+"%\n")

EVALUATION OF BASE MODEL:

Summary:
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.97       573
         1.0       0.91      0.73      0.81        94

    accuracy                           0.95       667
   macro avg       0.93      0.86      0.89       667
weighted avg       0.95      0.95      0.95       667

Confusion matrix:
[[566   7]
 [ 25  69]]

Accuracy:
95.2023988005997%

Precision:
90.78947368421053%

Recall:
73.40425531914893%

F1 Score:
81.17647058823529%

time: 35.8 ms (started: 2021-05-11 10:53:51 +00:00)
